In [13]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm

# Configuration du modèle et des hyperparamètres

Dans la cellule suivante, nous définissons les principaux hyperparamètres pour l'entraînement :
- Nombre d'époques d'entraînement
- Taille des batchs
- Patience pour l'early stopping
- Taux d'apprentissage
- Choix du device (GPU/CPU)


In [14]:

# Configuration
num_epochs = 20
batch_size = 32
patience = 3
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de {device}")

Utilisation de cpu


# Prétraitement des données et chargement du dataset

- Dans la cellule suivante, nous effectuons plusieurs opérations importantes :
- Configuration des transformations pour l'augmentation des données (redimensionnement, rotations, etc.)
- Chargement des images et création des étiquettes
- Split des données en ensembles d'entraînement, validation et test
- Définition d'une classe Dataset personnalisée pour charger les images

In [15]:


# Prétraitement
# transform = transforms.Compose([
#     transforms.Resize((128, 128)),  # Adapté pour le CNN (entrée 128x128 → sortie 16x16 après 3 poolings)
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),
#     transforms.RandomRotation(30),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])
transform_train = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])



In [16]:

# Dataset path
dataset_path = 'C:\\Users\\sebas\\PycharmProjects\\malaria\\data\\images'
parasitized_dir = os.path.join(dataset_path, 'Parasitized')
uninfected_dir = os.path.join(dataset_path, 'Uninfected')

# Fichiers et étiquettes
parasitized_files = [os.path.join(parasitized_dir, f) for f in os.listdir(parasitized_dir) if f.endswith('.png')]
uninfected_files = [os.path.join(uninfected_dir, f) for f in os.listdir(uninfected_dir) if f.endswith('.png')]
parasitized_labels = [0] * len(parasitized_files)
uninfected_labels = [1] * len(uninfected_files)

all_files = parasitized_files + uninfected_files
all_labels = parasitized_labels + uninfected_labels


# Split des données
 
Dans la cellule suivante, nous effectuons la séparation des données en trois ensembles :
- Un ensemble de test (20% des données)
- Un ensemble d'entraînement et de validation (80% des données), qui est ensuite divisé en :
- Un ensemble d'entraînement (64% du total)
- Un ensemble de validation (16% du total)
 
Nous utilisons un split stratifié pour conserver les proportions de chaque classe.
Nous définissons également une classe Dataset personnalisée pour charger les images.


In [17]:

# Split stratifié test (20%)
trainval_files, test_files, trainval_labels, test_labels = train_test_split(
    all_files, all_labels, test_size=0.2, stratify=all_labels, random_state=42
)

# Split stratifié val (20% de train_val)
train_files, val_files, train_labels, val_labels = train_test_split(
    trainval_files, trainval_labels, test_size=0.2, stratify=trainval_labels, random_state=42
)

# Dataset personnalisé
class MalariaDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


Dans la cellule suivante, nous définissons l'architecture de notre CNN personnalisé. Le modèle comprend :
- 3 couches de convolution avec des filtres de taille 3x3 et un padding de 1
- Des couches de pooling pour réduire la dimension spatiale
- Une couche fully connected avec 128 neurones
- Une couche de dropout pour éviter le surapprentissage 
- Une couche de sortie avec 2 neurones (classification binaire)


In [18]:
# Définition d'un modèle de réseau de neurones pour classer des images (cellule malade ou saine)
class CNNMalariaModel(nn.Module):
    def __init__(self, num_classes=2):  # On précise qu'on veut classer en 2 catégories (cellule malade ou saine)
        super(CNNMalariaModel, self).__init__()  # Initialisation du modèle à partir de la classe de base nn.Module

        # 1ère couche de convolution : elle regarde des petits morceaux de l'image grâce au kernel 3x3
        # Elle transforme les 3 canaux de couleur (rouge, vert, bleu) en 32 "cartes de caractéristiques"
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)  # Normalise les valeurs pour aider le réseau à apprendre plus vite

        # 2ème couche : prend les 32 cartes et en crée 64
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        # 3ème couche : transforme les 64 cartes en 128 cartes
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        # MaxPool : réduit la taille des images de moitié à chaque fois (comme un zoom arrière)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # GAP (Global Average Pooling) : réduit chaque carte à une seule valeur moyenne
        # Cela permet au modèle d’accepter des images de tailles différentes
        self.gap = nn.AdaptiveAvgPool2d((1, 1))

        # 1ère couche entièrement connectée : prend les 128 valeurs et en fait 128 nouvelles
        self.fc1 = nn.Linear(128, 128)

        # Dropout : coupe certaines connexions au hasard pendant l'entraînement (pour éviter que le réseau ne "triche")
        self.dropout = nn.Dropout(0.5)

        # Dernière couche : donne 2 valeurs, une pour chaque classe (ex : "malade" et "saine")
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):  # C’est ici qu’on décrit comment les données traversent le réseau
        # Étape 1 : première convolution + normalisation + activation (ReLU = garde les valeurs positives)
        x = self.pool(F.relu(self.bn1(self.conv1(x))))

        # Étape 2 : deuxième convolution + normalisation + activation + réduction de taille
        x = self.pool(F.relu(self.bn2(self.conv2(x))))

        # Étape 3 : troisième convolution + normalisation + activation + réduction de taille
        x = self.pool(F.relu(self.bn3(self.conv3(x))))

        # Réduction à une seule valeur par carte (grâce à GAP)
        x = self.gap(x)  # Résultat : un petit tableau de forme [batch, 128, 1, 1]

        # On "aplatie" ce petit tableau en une ligne pour le donner à la couche suivante
        x = x.view(x.size(0), -1)  # Devient [batch, 128]

        # Première couche complètement connectée avec ReLU
        x = F.relu(self.fc1(x))

        # Application du dropout (pendant l'entraînement uniquement)
        x = self.dropout(x)

        # Dernière couche qui donne 2 scores (un pour chaque classe)
        x = self.fc2(x)

        # Pas besoin d'ajouter Softmax ici : la fonction de perte CrossEntropy s'en occupe
        return x


Dans la cellule suivante, nous créons les datasets et dataloaders pour l'entraînement, la validation et le test.
Nous initialisons également le modèle, définissons la fonction de perte (CrossEntropyLoss) et l'optimiseur (Adam).
Nous implémentons aussi une fonction d'évaluation qui calcule la perte et l'exactitude sur un jeu de données.
Enfin, nous mettons en place la boucle d'entraînement avec early stopping pour éviter le surapprentissage.


In [19]:

# Datasets & Loaders
train_dataset = MalariaDataset(train_files, train_labels, transform=transform_train)
val_dataset = MalariaDataset(val_files, val_labels, transform=transform_train)
test_dataset = MalariaDataset(test_files, test_labels, transform=transform_train)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Modèle
model = CNNMalariaModel().to(device)

# Loss et optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


# Fonction d'évaluation
Dans la cellule suivante, nous définissons une fonction d'évaluation `evaluate()` qui permet de calculer la perte et l'exactitude du modèle sur un jeu de données donné. Cette fonction sera utilisée pour évaluer les performances du modèle sur les ensembles de validation et de test. Elle prend en paramètres le modèle et un dataloader, et retourne la perte moyenne et le pourcentage de prédictions correctes.


In [20]:

# Fonction d'évaluation
def evaluate(model, dataloader):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return running_loss / len(dataloader), 100 * correct / total


# Variables pour l'entraînement
- best_acc : stocke la meilleure exactitude obtenue sur l'ensemble de validation
- patience_counter : compte le nombre d'époques sans amélioration pour l'early stopping
- best_model_state : sauvegarde l'état du meilleur modèle
Nous créons également un dossier 'models' pour sauvegarder les checkpoints du modèle.


In [21]:

# Entraînement
best_acc = 0.0
patience_counter = 0
best_model_state = None
os.makedirs('models', exist_ok=True)


# Entraînement du modèle. Pour chaque époque :
- Nous calculons la perte et l'exactitude sur l'ensemble d'entraînement
- Nous évaluons le modèle sur l'ensemble de validation
- Nous sauvegardons le meilleur modèle si l'exactitude de validation s'améliore
- Nous appliquons l'early stopping si aucune amélioration n'est constatée pendant plusieurs époques
Une barre de progression tqdm affiche l'avancement de l'entraînement avec les métriques en temps réel.

In [ ]:

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    print(f'\nEpoch [{epoch+1}/{num_epochs}]')
    print('-' * 50)

    model.train()
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100 * correct / total:.2f}%'
        })

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total

    val_loss, val_acc = evaluate(model, val_loader)
    print(f'\nEpoch terminé: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

    if val_acc > best_acc:
        best_acc = val_acc
        best_model_state = model.state_dict()
        patience_counter = 0
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': best_model_state,
            'optimizer_state_dict': optimizer.state_dict(),
            'best_acc': best_acc,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
        }, 'models/best_model_cnn.pth')
        print(f'✅ Nouveau meilleur modèle sauvegardé avec une Val Acc de {best_acc:.2f}%')
    else:
        patience_counter += 1
        print(f'Patience: {patience_counter}/{patience}')
        if patience_counter >= patience:
            print(f'\n⏹️ Early stopping après {epoch + 1} epochs sans amélioration.')
            break

# Évaluation finale
print("\n📊 Évaluation finale sur le test set :")
model.load_state_dict(best_model_state)
test_loss, test_acc = evaluate(model, test_loader)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%')



Epoch [1/20]
--------------------------------------------------


Epoch 1/20:   0%|          | 0/552 [00:00<?, ?it/s]

Epoch 1/20: 100%|██████████| 552/552 [04:15<00:00,  2.16it/s, loss=1.0172, acc=82.42%]
